In [9]:
import subprocess
import os
import sys
sys.path.append('..//model_gen')
import setup_methods

working_directory = '/home/sebastian/Documents/GitHub/XenoFind/data/large_working_directory/'
ref_fasta = "/home/sebastian/Documents/GitHub/XenoFind/data/reads_large/230725_PZ_lib_v4_r10/fasta/good.fa"
working_subdir=''
fasta_path = ''
IS_TRAINING = True

if IS_TRAINING:
    fasta_path = ref_fasta
    working_subdir = 'model_training/'
else:
    working_subdir = 'model_testing/'
    fasta_path = working_directory + working_subdir + 'consensus.fa'
    
bam_path = working_directory + working_subdir + 'basecall_directory/aligned.bam'
pod5_path = working_directory + working_subdir + 'merged_pod5/merged.pod5'
json_dir = working_directory + working_subdir + 'json_files/'
parquet_dir = working_directory + working_subdir + 'parquet_superdirectory/'
batch_size = 100

# convert the json files to constituent parquet files based on batch size


In [ ]:
# Aggregate the reads to individual json files.
subprocess.run(['python',
                'aggregate_reads.py',
                '-v',
                '-y',
                '-bam',bam_path,
                '-pod5',pod5_path,
                '-fasta',fasta_path,
                '-output',json_dir])


In [ ]:
# convert the json files to batched parquet files
json_files = os.listdir(json_dir)

for file in json_files[:]:
    json_filepath = json_dir + file
    subdirectory = parquet_dir + file.split('.')[0] + '/'
    setup_methods.check_make_dir(subdirectory)
    subprocess.run(['python',
                    'feature_extraction.py',
                    #'--verbose',
                    '-batch_size', str(batch_size),
                    '-json_path', json_filepath,
                    '-output', subdirectory])
    

In [70]:
# load the parquet files to dataframes and combine to a list
parquet_subdirs = os.listdir(parquet_dir)

feature_read_list = []
for subdir in parquet_subdirs[:]:
    for parquet in os.listdir(parquet_dir + subdir):
        path = parquet_dir+subdir+"/"+parquet
        feature_read_list.append(pd.read_parquet(path))
        
assembled_df = pd.concat(feature_read_list).reset_index(drop=True)

In [72]:
assembled_df

,XNA_PRESENT,A,G,C,T,-,+,n,mm_prob,shentropy,...,std_sig_meds_w/o,std_sig_peaks_w/o,std_sig_time_w/o,std_sig_peaks_means_w/o,median_sig_means_w/o,median_sig_stds_w/o,median_sig_meds_w/o,median_sig_peaks_w/o,median_sig_time_w/o,median_sig_peaks_means_w/o
0,0,1.0,0.0,0.0,0.0,0.0,0.0,3,0.0,-0.000000,...,25.009094,375375.622879,0.000000,375375.622879,80.225230,10.867531,78.803474,643608.759397,0.002500,643608.759397
1,0,0.0,1.0,0.0,0.0,0.0,0.0,4,0.0,-0.000000,...,21.284302,422448.661502,0.000722,422448.661502,70.177572,3.763490,69.440685,302353.379448,0.001875,302353.379448
2,0,0.0,0.0,0.0,1.0,0.0,0.0,4,0.0,-0.000000,...,7.878848,45612.479358,0.000000,45612.479358,73.289830,10.327544,69.093914,134410.046566,0.001250,134410.046566
3,0,1.0,0.0,0.0,0.0,0.0,0.0,4,0.0,-0.000000,...,22.163045,512247.573202,0.000722,512247.573202,101.334851,15.318776,109.275883,555441.197725,0.001875,555441.197725
4,0,0.0,0.0,0.8,0.0,0.0,0.2,5,0.0,0.721928,...,7.181014,542902.076255,0.000625,542902.076255,104.828559,3.855856,104.551144,274727.594113,0.001250,274727.594113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,-0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2656,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,-0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2657,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,-0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2658,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,-0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
